In [143]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Libraries and Dataset

In [144]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, Normalizer, MaxAbsScaler

In [145]:
data = pd.read_csv('../input/hdb-flat-prices-19902021-march/ALL Prices 1990-2021 mar.csv')
data

# Dataset Overview

In [146]:
data.shape

In [147]:
data.nunique()

In [148]:
data.isnull().sum()

In [149]:
data.info()

fortunately there is no null data!

# Visualization

In [150]:
sns.histplot(data = data, x='resale_price', kde=True);

In [151]:
fig = plt.figure()
fig.set_size_inches(15, 6)
sns.countplot(data = data, x = 'year');

In [152]:
sns.countplot(data = data, y = 'flat_type');

In [153]:
fig = plt.figure()
fig.set_size_inches(8, 6)

sns.kdeplot(data[data['flat_type'] == '1 ROOM']['resale_price']);
sns.kdeplot(data[data['flat_type'] == '2 ROOM']['resale_price']);
sns.kdeplot(data[data['flat_type'] == '3 ROOM']['resale_price']);
sns.kdeplot(data[data['flat_type'] == '4 ROOM']['resale_price']);
sns.kdeplot(data[data['flat_type'] == '5 ROOM']['resale_price']);
sns.kdeplot(data[data['flat_type'] == 'EXECUTIVE']['resale_price']);
sns.kdeplot(data[data['flat_type'] == 'MULTI GENERATION']['resale_price']);

In [154]:
r1 = data['flat_type'].value_counts()[0]
r2 = data['flat_type'].value_counts()[1]
r3 = data['flat_type'].value_counts()[2]
r4 = data['flat_type'].value_counts()[3]
r5 = data['flat_type'].value_counts()[4]
r_Executive = data['flat_type'].value_counts()[5]
r_Multi = data['flat_type'].value_counts()[6]


fig = plt.figure()
fig.set_size_inches(8, 6)
plt.bar(x=['1 ROOM', '2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'EXECUTIVE', 'MULTI GENERATION'] , height = [r1, r2, r3, r4, r5, r_Executive, r_Multi]);

# Preprocessing

In [155]:
data.info()

In [156]:
print(data.columns)

In [157]:
data.columns = data.columns.str.strip()
data.drop(['month','street_name', 'address', 'price_psm_yearly', 'Core CPI', 'year_gni', 'flat_model','price cpi_adj', 'price_psm cpi_adj', 'bala lease pct', 'price_psm cpi_lease_adj implied'],axis=1,inplace=True)

In [158]:
data = data.drop(columns=['price lease_adj implied', 'price_psm lease_adj implied', 'price cpi_lease_adj implied','town','block','flat_type','storey_range' ])

In [159]:
data.info()

In [160]:
X = data.drop(columns=['resale_price'])
y = data['resale_price']

In [161]:
X.info()

In [162]:
y.dtypes

In [163]:
X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=111)
X_train2 , X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.15)
X_train3 , X_test3, y_train3, y_test3 = train_test_split(X, y, test_size=0.15)
X_train4,X_test4,y_train4,y_test4 = train_test_split(X,y,test_size = 0.15)

In [164]:
X_train.shape, y_train.shape

In [165]:
X_train2.shape, y_train2.shape

In [166]:
X_test.shape, y_test.shape

# Regression Modeling

### Linear Regression

In [167]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.decomposition import PCA

In [168]:
from sklearn.linear_model import LinearRegression
LR = LinearRegression()
LR.fit(X, y)

In [169]:
y_pred = LR.predict(X)
plt.scatter(y_pred,y)

In [170]:
LR.score(X,y) #R2

In [171]:
steps = [('scaler',StandardScaler()),
         ('poly features',PolynomialFeatures()),
         ('dimension reduction',PCA(n_components=0.96)),
         ('Regression',LinearRegression())]
lr_pipeline = Pipeline(steps = steps)

In [172]:
lr_pipeline.fit(X,y)

In [173]:
lr_pipeline.score(X,y)

In [174]:
plt.scatter(lr_pipeline.predict(X),y)

### Ridge Regression

In [175]:
from sklearn.linear_model import Ridge
from numpy.linalg import norm

In [176]:
rr = Ridge(alpha = 0.1).fit(X, y)
rr.score(X, y)

In [177]:
norm(rr.coef_)

In [178]:
steps = [('scaler',StandardScaler()),
         ('poly features',PolynomialFeatures(degree = 3,interaction_only = True)),
         ('dimension reduction',PCA(n_components=0.96)),
         ('Regression',Ridge(alpha=0.9))]

Ridge_pipeline = Pipeline(steps = steps)
Ridge_pipeline.fit(X_train2,y_train2)
Ridge_pipeline.score(X_test2,y_test2)

In [179]:
y_pred2 = rr.predict(X_test2)
plt.scatter(y_pred2,y_test2)

### Lasso Regression

In [180]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold,cross_validate

In [181]:
ls = Lasso(alpha=0.01).fit(X_train3,y_train3)
ls.score(X_test3,y_test3)

In [182]:
ls.coef_

In [183]:
norm(ls.coef_,1)

In [184]:
y_pred3 = ls.predict(X_test3)
plt.scatter(y_pred3,y_test3)

## Elastic Net Regression

In [185]:
from sklearn.linear_model import ElasticNet

In [186]:
from sklearn.model_selection import LeaveOneOut

In [187]:
en = ElasticNet(alpha=0.5,l1_ratio=0.3).fit(X_train4,y_train4)
en.score(X_test4,y_test4)

In [188]:
en.coef_

In [189]:
y_pred4 = LR.predict(X_test4)
plt.scatter(y_pred4,y_test4)